In [1]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = 'plotly_white'

import pandas as pd
import numpy as np

In [2]:
coefs_df_real_xgb = pd.read_csv(f'plots/fv/a_tilde_fv_XGB.csv')
coefs_df_real_NN = pd.read_csv(f'plots/fv/a_tilde_fv_NN.csv')

In [3]:
cols = coefs_df_real_xgb.columns

In [4]:
res_df_fv_NN = pd.read_csv(f'plots/fv/res_df_fv_NN.csv')
res_df_fv_XGB = pd.read_csv(f'plots/fv/res_df_fv_XGB.csv')

In [5]:
energies = [0.1, 0.3, 0.6] + list(range(1, 10))
energies = [round(1.022+i, 5) for i in energies]

In [6]:
accidentals = pd.read_csv('plots/fv/accidentals.csv')

In [7]:
R_min = 16.8
R_max = 17.2
N_points = 101
Rs = np.linspace(R_min, R_max, N_points)

def list_with_params_dict(coefs_df, res_df):
    params_list = []
    for i in range(res_df.shape[0]):
        params_dict = dict()
        params_dict['frac'] = dict(events=[res_df['f'][i], res_df['df'][i]],
                                accidentals=[accidentals['accidentals'][i], accidentals['accidentals_err'][i]])
        params_dict['R'] = Rs[i]
        params_dict['eres_abc'] = dict(a=coefs_df[cols[0]][i],
                                    b=coefs_df[cols[2]][i],
                                    c=coefs_df[cols[4]][i],
                                    a_err=coefs_df[cols[1]][i],
                                    b_err=coefs_df[cols[3]][i],
                                    c_err=coefs_df[cols[5]][i],
                                    r_ab=coefs_df[cols[10]][i],
                                    r_ac=coefs_df[cols[11]][i],
                                    r_bc=coefs_df[cols[12]][i])
        params_dict['eres'] = dict(resolution=res_df.loc[i][:24][::2].to_list(),
                                resolution_err=res_df.loc[i][:24][1::2].to_list(),
                               energies=energies)

        params_list.append(params_dict)
        
    return params_list

In [8]:
import pickle

xgb_list = list_with_params_dict(coefs_df_real_xgb, res_df_fv_XGB)
nn_list = list_with_params_dict(coefs_df_real_NN, res_df_fv_NN)
result_dict = dict(xgboost=xgb_list, neural_network=nn_list)

with open('plots/fv/reconstruction_params.pkl', 'wb') as handle:
    pickle.dump(result_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
Rs = np.linspace(R_min, R_max, N_points)

color_xgb = 'rgba(150, 0, 0, 0.3)'
color_NN = 'rgba(0, 0, 200, 0.3)'

In [19]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=Rs,
        y=coefs_df_real_xgb[r'$\tilde{a}$']+coefs_df_real_xgb[r'$\Delta \tilde{a}$'],
        line=dict(color='black', width=0.75),
        showlegend=False
    )
)

fig.add_trace(
    go.Scatter(
        x=Rs,
        y=coefs_df_real_xgb[r'$\tilde{a}$']-coefs_df_real_xgb[r'$\Delta \tilde{a}$'],
        fill='tonexty',
        fillcolor=color_xgb,
        name='XGB',
        line=dict(color='black', width=0.75)
    )
)

fig.add_trace(
    go.Scatter(
        x=Rs,
        y=coefs_df_real_xgb[r'$\tilde{a}$'],
        mode='lines',#'lines+markers',
        showlegend=False,
        line=dict(
            dash='dash',
            color='black'
        ),
        marker=dict(
            color='black',
            size=8,
            symbol='x'
        ),
    )
)

fig.add_trace(
    go.Scatter(
        x=Rs,
        y=coefs_df_real_NN[r'$\tilde{a}$']+coefs_df_real_NN[r'$\Delta \tilde{a}$'],
        line=dict(color='black', width=0.75),
        showlegend=False
    )
)

fig.add_trace(
    go.Scatter(
        x=Rs,
        y=coefs_df_real_NN[r'$\tilde{a}$']-coefs_df_real_NN[r'$\Delta \tilde{a}$'],
        fill='tonexty',
        fillcolor=color_NN,
        name='NN',
        line=dict(color='black', width=0.75)
    )
)

fig.add_trace(
    go.Scatter(
        x=Rs,
        y=coefs_df_real_NN[r'$\tilde{a}$'],
        mode='lines',#'lines+markers',
        showlegend=False,
        line=dict(
            dash='dash',
            color='black'
        ),
        marker=dict(
            color='black',
            size=8,
            symbol='x'
        ),
    )
)

xaxis = lambda range: dict(
    showline=True,
    ticks='outside',
    mirror=True,
    tick0=1,
    dtick=0.2,
    range=range,
    linecolor='black',
    showgrid=True,
    gridcolor='grey',
    gridwidth=0.25,
)

yaxis = lambda range: dict(
    showline=True,
    ticks='outside',
    mirror=True,
    linecolor='black',
    range=range,
    showgrid=True,
    gridcolor='grey',
    gridwidth=0.25,
    zeroline=True,
    zerolinecolor='black',
    zerolinewidth=0.25
)

fig.update_layout(
    xaxis_title="R, m",
    yaxis_title=r'$\tilde{a}$',
    xaxis = xaxis([R_min, R_max]),
    yaxis = yaxis([2.76, 2.96]),#[2.73, 2.94]),
    legend=dict(
        x=0.01,
        y=0.99,
        title_font_family="Times New Roman",
        font=dict(
            family="Times New Roman",
            size=18,
            color="black"
        ),
        bordercolor="Black",
        borderwidth=2
    ),
    showlegend=True,
    height=500,
    width=950,
    font=dict(
            family="Times New Roman",
            size=16,
    ),
)

fig.show()
pio.write_image(fig, 'plots/fv/atilde_vs_R.pdf', height=500, width=950)